# Drone Routing Optimization Problem

## Model Initialization

In [1]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp
import csv
import numpy as np
#import pandas as pd
#import time
#from datetime import date

# initialize model object
drone_model = glp.Solver('Drone Routing Problem 2', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Parameters

### Import distance matrix

In [2]:
# import csv distance matrix as list
with open('Orders_Data.csv', newline='') as f:
    reader = csv.reader(f)
    distance = list(reader)

# remove blanks
for i in range(len(distance)):
    distance[i] = [item for item in distance[i] if item]
distance = [item for item in distance if item]
distance = np.array(distance, dtype = 'float')

# Cost per unit of distance
truck_cost_per_dist = 10.0
drone_cost_per_dist = 3.0

# Create cost matrices for truck & drone
truck_cost = distance * truck_cost_per_dist
drone_cost = distance * drone_cost_per_dist
truck_cost = truck_cost.tolist()
drone_cost = drone_cost.tolist()

In [3]:
distance.tolist()

[[0.0, 100.0, 81.0, 99.0, 61.0, 111.0],
 [100.0, 0.0, 97.0, 98.0, 41.0, 93.0],
 [81.0, 97.0, 0.0, 19.0, 66.0, 36.0],
 [99.0, 98.0, 19.0, 0.0, 74.0, 18.0],
 [61.0, 41.0, 66.0, 74.0, 0.0, 77.0],
 [111.0, 93.0, 36.0, 18.0, 77.0, 0.0]]

### Initialize Model Parameters

In [4]:
# model parameters
R = list(range(1, len(distance))) # Nodes excluding origin & final 'dummy' node (1 to 10 index)
N = list(range(len(distance))) # All nodes (0 to 11 index)
M = len(distance) + 100 # Large number
D = list(range(3)) # Drones onboard the truck (0 to 2 index)

In [5]:
R

[1, 2, 3, 4, 5]

In [6]:
N

[0, 1, 2, 3, 4, 5]

## Decision Variables

In [7]:
# Truck path from node i to j
dimensions = (int(len(N)), int(len(N))) # Create truck arc dimensions - 11 x 11 (10 x 10 index)
truck_arc = np.zeros(dimensions).tolist() # create nested list
for i in N:
    for j in N:
        if i !=j:
            truck_arc[i][j] = drone_model.IntVar(0, 1, str(i) + "." + str(j)) #create binary variable for each possible truck path
truck_arc

[[0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
 [1.0, 0.0, 1.2, 1.3, 1.4, 1.5],
 [2.0, 2.1, 0.0, 2.3, 2.4, 2.5],
 [3.0, 3.1, 3.2, 0.0, 3.4, 3.5],
 [4.0, 4.1, 4.2, 4.3, 0.0, 4.5],
 [5.0, 5.1, 5.2, 5.3, 5.4, 0.0]]

In [8]:
# The kth drone path from node i to j
dimensions = (int(len(N)), int(len(N))) # Create drone arc dimensions - 12 x 12 x 3 (index 11 x 11 x 2)
drone_arc = np.zeros(dimensions).tolist() # create nested list

for i in R:
    for j in R:
        if i != j:
            drone_arc[i][j] = drone_model.IntVar(0, 1, str(i) + "." + str(j)) #create binary variable for each possible drone path

        #for k in D: [k]  + "." + str(k) , int(len(D))
        
# All drone paths from node 2 (index 1)
drone_arc

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.2, 1.3, 1.4, 1.5],
 [0.0, 2.1, 0.0, 2.3, 2.4, 2.5],
 [0.0, 3.1, 3.2, 0.0, 3.4, 3.5],
 [0.0, 4.1, 4.2, 4.3, 0.0, 4.5],
 [0.0, 5.1, 5.2, 5.3, 5.4, 0.0]]

In [9]:
# create position variable (lambda / u)
pos = list(N)
for i in N:
    pos[i] = drone_model.IntVar(0, len(N), "p." + str(i))
pos

[p.0, p.1, p.2, p.3, p.4, p.5]

In [10]:
len(N)

6

## Objective

In [11]:
# create objective function
optimal_route = drone_model.Objective()
optimal_route.SetMinimization() # Minimize cost

for i in N:
    for j in N:
        if i != j: 
            optimal_route.SetCoefficient(truck_arc[i][j], truck_cost[i][j]) # Truck cost per arc
            print(str(i) + '.' + str(j))

0.1
0.2
0.3
0.4
0.5
1.0
1.2
1.3
1.4
1.5
2.0
2.1
2.3
2.4
2.5
3.0
3.1
3.2
3.4
3.5
4.0
4.1
4.2
4.3
4.5
5.0
5.1
5.2
5.3
5.4


In [12]:
for i in R:
    for j in R:
        if i != j: 
            optimal_route.SetCoefficient(drone_arc[i][j], drone_cost[i][j]) # Drone cost per arc
            print(str(i) + '.' + str(j))

1.2
1.3
1.4
1.5
2.1
2.3
2.4
2.5
3.1
3.2
3.4
3.5
4.1
4.2
4.3
4.5
5.1
5.2
5.3
5.4


## Constraints

In [13]:
first_arc = drone_model.Constraint(1, 1)
for j in R:
    first_arc.SetCoefficient(truck_arc[0][j], 1)
    print(str(j))

1
2
3
4
5


In [14]:
last_arc = drone_model.Constraint(1, 1)
for i in R:
    last_arc.SetCoefficient(truck_arc[i][0], 1)
    print(str(i))

1
2
3
4
5


In [15]:
bal = list(N)
for j in N:
    bal[j] = drone_model.Constraint(0, 0)
    for i in N:
        if i != j:
            bal[j].SetCoefficient(truck_arc[i][j], 1)
            bal[j].SetCoefficient(truck_arc[j][i], -1)
            print(str(i) + '.' + str(j))

1.0
2.0
3.0
4.0
5.0
0.1
2.1
3.1
4.1
5.1
0.2
1.2
3.2
4.2
5.2
0.3
1.3
2.3
4.3
5.3
0.4
1.4
2.4
3.4
5.4
0.5
1.5
2.5
3.5
4.5


In [17]:
num_constr = []
dimensions = (int(len(N)), int(len(N))) 
launch = np.zeros(dimensions).tolist()
for i in R:
    for j in R:
        if i != j:
            launch[i][j] = drone_model.Constraint(-drone_model.infinity(), 0)
            launch[i][j].SetCoefficient(drone_arc[i][j], 1)
            num_constr.append(str(i) + '.' + str(j))
            for k in N:
                if k != i:
                    launch[i][j].SetCoefficient(truck_arc[k][i], -1)
                    print('drone: ', str(i), '.', str(j), ' truck: ', str(k), '.', str(i), sep = '')
            print('')

drone: 1.2 truck: 0.1
drone: 1.2 truck: 2.1
drone: 1.2 truck: 3.1
drone: 1.2 truck: 4.1
drone: 1.2 truck: 5.1

drone: 1.3 truck: 0.1
drone: 1.3 truck: 2.1
drone: 1.3 truck: 3.1
drone: 1.3 truck: 4.1
drone: 1.3 truck: 5.1

drone: 1.4 truck: 0.1
drone: 1.4 truck: 2.1
drone: 1.4 truck: 3.1
drone: 1.4 truck: 4.1
drone: 1.4 truck: 5.1

drone: 1.5 truck: 0.1
drone: 1.5 truck: 2.1
drone: 1.5 truck: 3.1
drone: 1.5 truck: 4.1
drone: 1.5 truck: 5.1

drone: 2.1 truck: 0.2
drone: 2.1 truck: 1.2
drone: 2.1 truck: 3.2
drone: 2.1 truck: 4.2
drone: 2.1 truck: 5.2

drone: 2.3 truck: 0.2
drone: 2.3 truck: 1.2
drone: 2.3 truck: 3.2
drone: 2.3 truck: 4.2
drone: 2.3 truck: 5.2

drone: 2.4 truck: 0.2
drone: 2.4 truck: 1.2
drone: 2.4 truck: 3.2
drone: 2.4 truck: 4.2
drone: 2.4 truck: 5.2

drone: 2.5 truck: 0.2
drone: 2.5 truck: 1.2
drone: 2.5 truck: 3.2
drone: 2.5 truck: 4.2
drone: 2.5 truck: 5.2

drone: 3.1 truck: 0.3
drone: 3.1 truck: 1.3
drone: 3.1 truck: 2.3
drone: 3.1 truck: 4.3
drone: 3.1 truck: 5.3

d

In [18]:
coverage = list(N)
for j in R:
    coverage[j] = drone_model.Constraint(1, 1)
    for i in N:
        if i != j:
            coverage[j].SetCoefficient(truck_arc[i][j], 1)
            print('truck: ', str(i), '.', str(j), sep = '')
    print('')
    for i in R:
        if i != j:
            coverage[j].SetCoefficient(drone_arc[i][j], 1)
            print('drone: ', str(i), '.', str(j), sep = '')
    print('')

truck: 0.1
truck: 2.1
truck: 3.1
truck: 4.1
truck: 5.1

drone: 2.1
drone: 3.1
drone: 4.1
drone: 5.1

truck: 0.2
truck: 1.2
truck: 3.2
truck: 4.2
truck: 5.2

drone: 1.2
drone: 3.2
drone: 4.2
drone: 5.2

truck: 0.3
truck: 1.3
truck: 2.3
truck: 4.3
truck: 5.3

drone: 1.3
drone: 2.3
drone: 4.3
drone: 5.3

truck: 0.4
truck: 1.4
truck: 2.4
truck: 3.4
truck: 5.4

drone: 1.4
drone: 2.4
drone: 3.4
drone: 5.4

truck: 0.5
truck: 1.5
truck: 2.5
truck: 3.5
truck: 4.5

drone: 1.5
drone: 2.5
drone: 3.5
drone: 4.5



In [19]:
dimensions = (int(len(N)), int(len(N))) # Create MTZ constraint dimensions
MTZ = np.zeros(dimensions).tolist() # create nested list

for i in R:
    for j in R:
        if i !=j:
            MTZ[i][j] = drone_model.Constraint(-drone_model.infinity(), len(N)-2)
            MTZ[i][j].SetCoefficient(truck_arc[i][j], len(N)-1)
            MTZ[i][j].SetCoefficient(pos[i], 1)
            MTZ[i][j].SetCoefficient(pos[j], -1)
            print('pos', i, ' - ', 'pos', j, ' + ', len(N)-1,' * ', '(truck ', i,'.' ,j, ') <= ',  len(N)-2, sep = '')

pos1 - pos2 + 5 * (truck 1.2) <= 4
pos1 - pos3 + 5 * (truck 1.3) <= 4
pos1 - pos4 + 5 * (truck 1.4) <= 4
pos1 - pos5 + 5 * (truck 1.5) <= 4
pos2 - pos1 + 5 * (truck 2.1) <= 4
pos2 - pos3 + 5 * (truck 2.3) <= 4
pos2 - pos4 + 5 * (truck 2.4) <= 4
pos2 - pos5 + 5 * (truck 2.5) <= 4
pos3 - pos1 + 5 * (truck 3.1) <= 4
pos3 - pos2 + 5 * (truck 3.2) <= 4
pos3 - pos4 + 5 * (truck 3.4) <= 4
pos3 - pos5 + 5 * (truck 3.5) <= 4
pos4 - pos1 + 5 * (truck 4.1) <= 4
pos4 - pos2 + 5 * (truck 4.2) <= 4
pos4 - pos3 + 5 * (truck 4.3) <= 4
pos4 - pos5 + 5 * (truck 4.5) <= 4
pos5 - pos1 + 5 * (truck 5.1) <= 4
pos5 - pos2 + 5 * (truck 5.2) <= 4
pos5 - pos3 + 5 * (truck 5.3) <= 4
pos5 - pos4 + 5 * (truck 5.4) <= 4


In [20]:
position = drone_model.Constraint(0, 0)
position.SetCoefficient(pos[0], 1)

In [ ]:
# Solve the model and print optimal solution
#start_time = time.time()
status = drone_model.Solve()                 # solve drone_model and display the solution

print('Solution Status =', status)
print('Number of variables =', drone_model.NumVariables())
print('Number of constraints =', drone_model.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
#print('Optimal Value = %.2f' % optimal_route.Value())

#print("Solution Time: %.2f seconds" % (time.time() - start_time))
#runtime = round(time.time() - start_time)
#today = str(date.today())

# Display optimal solution
#for i in N:
#    print('pos[%d] = %d' % (i, pos[i].solution_value()))
#    for j in N:
#        if truck_arc[i][j].solution_value() == 1:
#            print('Truck ', i, ' to ', j, ': ', truck_arc[i][j].solution_value(), sep = '')
#for i in R:
#    for j in R:
#        if drone_arc[i][j].solution_value() == 1:
#                print('Drone from ', i, ' to ', j, ': ', drone_arc[i][j].solution_value(), sep = '')
        #for k in D:

In [ ]:
import gc
gc.collect()